<a href="https://colab.research.google.com/github/MrRelu/project/blob/main/Possible_Overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dataset_path = kagglehub.dataset_download('muhammadqasimshabbir/amini-soil-prediction-challenge-dataset')

print('Data source import complete.')

Data source import complete.


In [2]:
!pip install optuna fpdf xgboost catboost optuna-integration

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.6 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=94fe5c0e2dc393210292cc69cb044648935820aba753556233e3af15291b0ad4
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [4]:
# Load datasets
train_df = pd.read_csv(dataset_path+'/Train.csv')
test_df = pd.read_csv(dataset_path+'/Test.csv')
train_gap_df = pd.read_csv(dataset_path+'/Gap_Train.csv')
test_gap_df1 = pd.read_csv(dataset_path+'/Gap_Test.csv')
sample_submission = pd.read_csv(dataset_path+'/SampleSubmission.csv')
test_gap_df = pd.merge(test_gap_df1, test_df[['PID', 'BulkDensity']], on='PID')

In [5]:
# prompt: input missing values in train_df and test_df with the mean, only do it for columns that have missing values

# Fill missing values with the mean for columns with missing values in train_df
for column in train_df.columns:
  if train_df[column].isnull().any():
    train_df[column].fillna(train_df[column].mean(), inplace=True)

# Fill missing values with the mean for columns with missing values in test_df
for column in test_df.columns:
  if test_df[column].isnull().any():
    test_df[column].fillna(test_df[column].mean(), inplace=True)

In [ ]:

"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostRegressor, Pool
import optuna
from optuna.pruners import SuccessiveHalvingPruner
# from optuna.integration import CatBoostPruningCallback
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import pickle
from fpdf import FPDF

output_path="/content/drive/MyDrive"

# Define targets and corresponding level columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test set
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# Define Optuna objective function
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 11),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "loss_function": "RMSE",
        "early_stopping_rounds": 50,
        "random_seed": 42,
        "verbose": False,
    }

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)

    model = CatBoostRegressor(**params,)

    model.fit(train_pool, eval_set=val_pool)
    preds = model.predict(val_pool)
    return np.sqrt(mean_squared_error(y_val, preds))

# Containers to track results
final_predictions = pd.DataFrame({'PID': test_ids})
cv_scores = {}
best_params_dict = {}  # To store best parameters per target

# Loop over each target variable
for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]
    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    # Split for tuning
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=stratify_vals, random_state=42
    )

    # Tune with Optuna
    pruner = SuccessiveHalvingPruner()
    study = optuna.create_study(direction="minimize", pruner=pruner)
    for _ in tqdm(range(20), desc=f"Tuning {target}"):
        study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=1)

    best_params = study.best_trial.params
    print(f"Best Parameters for {target}: {best_params}")
    best_params_dict[target] = best_params  # Save for export

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        train_pool = Pool(X_tr, y_tr)
        val_pool = Pool(X_vl, y_vl)

        model = CatBoostRegressor(
            iterations=1000,
            early_stopping_rounds=50,
            loss_function="RMSE",
            random_seed=42,
            verbose=False,
            **best_params
        )

        model.fit(train_pool, eval_set=val_pool)

        val_preds = model.predict(val_pool)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    final_predictions[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Overall score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# Save submission
final_predictions.to_csv(output_path+"/Cat_submission.csv", index=False)

# Save best parameters to pickle file
with open(output_path+"/best_catboost_params.pkl", "wb") as f:
    pickle.dump(best_params_dict, f)

# Save best parameters to PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Best CatBoost Parameters per Target", ln=True, align="C")
pdf.ln(10)

for target, params in best_params_dict.items():
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(200, 10, txt=f"{target}:", ln=True)
    pdf.set_font("Arial", size=11)
    for key, value in params.items():
        pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
    pdf.ln(5)

pdf.output(output_path+"/best_catboost_params.pdf")"""

In [ ]:
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostRegressor, Pool

dir_with_params= './'

with open(dir_with_params+"best_catboost_params.pkl", "rb") as f:
    best_param_dict = pickle.load(f)
# Containers to track results
# Define targets and corresponding level columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test set
X_test = test_df.drop(columns=['PID', 'site',])
test_ids = test_df['PID']
CAT = pd.DataFrame({'PID': test_ids})
cv_scores = {}
best_params_dict = {}  # To store best parameters per target

# Loop over each target variable
for target in target_columns:
    best_params=best_params_dict[target]
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]
    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        train_pool = Pool(X_tr, y_tr)
        val_pool = Pool(X_vl, y_vl)

        model = CatBoostRegressor(
            iterations=1000,
            early_stopping_rounds=50,
            loss_function="RMSE",
            random_seed=42,
            verbose=False,
            **best_params
        )

        model.fit(train_pool, eval_set=val_pool)

        val_preds = model.predict(val_pool)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    CAT[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Overall score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# Save submission
CAT.to_csv("Cat_submission.csv", index=False)

In [7]:
"""
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
from optuna.integration import XGBoostPruningCallback
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("xgboost").setLevel(logging.ERROR)

# Define targets and corresponding level columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test set
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# Containers to track results
final_predictions = pd.DataFrame({'PID': test_ids})
cv_scores = {}
best_params_dict = {}

# Loop over each target variable
for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]

    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    # Split for tuning
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=stratify_vals, random_state=42
    )

    # Define Optuna objective
    def objective(trial):
        params = {
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "eval_metric": "rmse",
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "lambda": trial.suggest_float("lambda", 0.0, 5.0),
            "alpha": trial.suggest_float("alpha", 0.0, 5.0),
            "n_estimators": 1000,
            "random_state": 42,
            "early_stopping_rounds":50

        }

        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            #callbacks=[
               # XGBoostPruningCallback(trial, "validation_0-rmse"),
                #xgb.callback.EarlyStopping(rounds=50)
           # ],
            verbose=False
        )
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    # Run Optuna tuning
    study = optuna.create_study(direction="minimize")
    for _ in tqdm(range(20), desc=f"Tuning {target}"):
        study.optimize(objective, n_trials=1)

    best_params = study.best_trial.params
    best_params.update({
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "eval_metric": "rmse",
        "n_estimators": 1000,
        "random_state": 42,
    })
    print(f"Best Parameters for {target}: {best_params}")
    best_params_dict[target] = best_params

    # Cross-validation with best params
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = xgb.XGBRegressor(**best_params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_vl, y_vl)],
           # callbacks=[
               # xgb.callback.EarlyStopping(rounds=50)
          #  ],
            verbose=False
        )

        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    final_predictions[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Overall score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# Save submission
final_predictions.to_csv("/content/drive/MyDrive/No_lon_XGb_submission.csv", index=False)
# Save best parameters to pickle file
with open("/content/drive/MyDrive/best_xgb_params.pkl", "wb") as f:
    pickle.dump(best_params_dict, f)

# Save best parameters to PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Best XGBOOST Parameters per Target", ln=True, align="C")
pdf.ln(10)

for target, params in best_params_dict.items():
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(200, 10, txt=f"{target}:", ln=True)
    pdf.set_font("Arial", size=11)
    for key, value in params.items():
        pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
    pdf.ln(5)

pdf.output("/content/drive/MyDrive/best_xgb_params.pdf") """

'\nfrom sklearn.model_selection import train_test_split, StratifiedKFold\nfrom sklearn.metrics import mean_squared_error\nimport xgboost as xgb\nimport optuna\nfrom optuna.integration import XGBoostPruningCallback\nfrom tqdm import tqdm\nimport warnings\nwarnings.filterwarnings("ignore")\nimport logging\nlogging.getLogger("xgboost").setLevel(logging.ERROR)\n\n# Define targets and corresponding level columns\ntarget_columns = [\'N\', \'P\', \'K\', \'Ca\', \'Mg\', \'S\', \'Fe\', \'Mn\', \'Zn\', \'Cu\', \'B\']\nlevel_columns = {target: f"{target}_level" for target in target_columns}\n\n# Prepare test set\nX_test = test_df.drop(columns=[\'PID\', \'site\'])\ntest_ids = test_df[\'PID\']\n\n# Containers to track results\nfinal_predictions = pd.DataFrame({\'PID\': test_ids})\ncv_scores = {}\nbest_params_dict = {}\n\n# Loop over each target variable\nfor target in target_columns:\n    print(f"\n=== Processing Target: {target} ===")\n    level_col = level_columns[target]\n\n    drop_cols = targe

In [ ]:

import xgboost as xgb
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
dir_with_params= './'

# Load best parameters
with open(dir_with_params+"best_xgb_params.pkl", "rb") as f:
    best_params_dict = pickle.load(f)

# Target columns and level mapping
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test data
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# Prepare output container
XGB = pd.DataFrame({'PID': test_ids})
cv_scores = {}

# Loop over each target
for target in target_columns:
    print(f"\n=== Retraining Target: {target} ===")
    level_col = level_columns[target]

    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")

        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = xgb.XGBRegressor(**best_params_dict[target])
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_vl, y_vl)],
            verbose=False
        )

        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f" Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    XGB[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Save predictions
XGB.to_csv("/content/drive/MyDrive/Retrained_XGB_submission.csv", index=False)

# Show overall RMSE
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

In [ ]:
"""
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
from optuna.pruners import SuccessiveHalvingPruner
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("lightgbm").setLevel(logging.ERROR)

# Define targets and corresponding level columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test set
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# Containers to track results
final_predictions = pd.DataFrame({'PID': test_ids})
cv_scores = {}
best_params_dict = {}

# Loop over each target variable
for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]

    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    # Split for tuning
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=stratify_vals, random_state=42
    )

    # Define Optuna objective
    def objective(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 16, 64),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
            "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
            "n_estimators": trial.suggest_int("n_estimators",300, 1200),
            "random_state": 42,
        }

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val,y_val)],
            callbacks=[LightGBMPruningCallback(trial, "l2")],
            #verbose=False
        )
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse

    # Run Optuna tuning
    pruner = SuccessiveHalvingPruner()
    study = optuna.create_study(direction="minimize", pruner=pruner)

    for _ in tqdm(range(60), desc=f"Tuning {target}"):
        study.optimize(objective, n_trials=2)

    best_params = study.best_trial.params
    best_params.update({
        "n_estimators": 1000,
        "random_state": 42,
    })
    print(f"Best Parameters for {target}: {best_params}")
    best_params_dict[target] = best_params

    # Cross-validation with best params
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(**best_params)
        model.fit(
            X_tr,y_tr,
            eval_set=[(X_vl, y_vl)],
            callbacks=[lgb.early_stopping(stopping_rounds=50)],
            #verbose=False
        )

        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    final_predictions[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Overall score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# Save submission
final_predictions.to_csv("/content/drive/MyDrive/best_lgb_submission.csv", index=False)
# Save best parameters to pickle file
with open("/content/drive/MyDrive/best_lgb_params.pkl", "wb") as f:
    pickle.dump(best_params_dict, f)

# Save best parameters to PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Best LGB Parameters per Target", ln=True, align="C")
pdf.ln(10)

for target, params in best_params_dict.items():
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(200, 10, txt=f"{target}:", ln=True)
    pdf.set_font("Arial", size=11)
    for key, value in params.items():
        pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
    pdf.ln(5)

pdf.output("/content/drive/MyDrive/best_lgb_params.pdf") """

In [ ]:

import lightgbm as lgb
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

dir_with_params= './'

# === Load best parameters ===
with open(dir_with_params+"best_lgb_params.pkl", "rb") as f:
    best_params_dict = pickle.load(f)

# === Define targets and their level mappings ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# === Prepare test set ===
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# === Prediction containers ===
LGB = pd.DataFrame({'PID': test_ids})
cv_scores = {}

# === Loop through each target ===
for target in target_columns:
    print(f"\n=== Retraining for Target: {target} ===")
    level_col = level_columns[target]

    # === Prepare training data ===
    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")

        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(**best_params_dict[target])
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_vl, y_vl)],
            callbacks=[lgb.early_stopping(stopping_rounds=50)],
            verbose=False
        )

        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    LGB[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# === Overall score ===
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# === Save predictions ===
LGB.to_csv("/content/drive/MyDrive/Final_LGB_Retrained_Submission.csv", index=False)

In [8]:
"""
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import optuna
from tqdm import tqdm


# Define targets and corresponding level columns
target_columns = ['Ca', 'P', 'K', 'N', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# Prepare test set
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# Define Optuna objective function for Random Forest
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 20, 30),
       # "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
       # "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        #"max_features": trial.suggest_categorical("max_features", ["log2", "sqrt", None]),
        "random_state": 42,
        "n_jobs": -1
    }

    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

# Containers to track results
final_predictions = pd.DataFrame({'PID': test_ids})
cv_scores = {}
best_params_dict = {}

# Loop over each target variable
for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]
    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    # Split for tuning
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=stratify_vals, random_state=42
    )

    # Optuna tuning
    study = optuna.create_study(direction="minimize")
    for _ in tqdm(range(20), desc=f"Tuning {target}"):
        study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=1)

    best_params = study.best_trial.params
    print(f"Best Parameters for {target}: {best_params}")
    best_params_dict[target] = best_params

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")
        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = RandomForestRegressor(
            **best_params,
            random_state=42,
            n_jobs=-1
        )

        model.fit(X_tr, y_tr)
        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    final_predictions[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# Overall score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

# Save submission
final_predictions.to_csv("/content/drive/MyDrive/submission_rf.csv", index=False)
# Save best parameters to pickle file
with open("/content/drive/MyDrive/best_Rand_params.pkl", "wb") as f:
    pickle.dump(best_params_dict, f)

# Save best parameters to PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Best CatBoost Parameters per Target", ln=True, align="C")
pdf.ln(10)

for target, params in best_params_dict.items():
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(200, 10, txt=f"{target}:", ln=True)
    pdf.set_font("Arial", size=11)
    for key, value in params.items():
        pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
    pdf.ln(5)

pdf.output("/content/drive/MyDrive/best_Rand_params.pdf") """

'\nfrom sklearn.model_selection import train_test_split, StratifiedKFold\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.metrics import mean_squared_error\nimport optuna\nfrom tqdm import tqdm\n\n\n# Define targets and corresponding level columns\ntarget_columns = [\'Ca\', \'P\', \'K\', \'N\', \'Mg\', \'S\', \'Fe\', \'Mn\', \'Zn\', \'Cu\', \'B\']\nlevel_columns = {target: f"{target}_level" for target in target_columns}\n\n# Prepare test set\nX_test = test_df.drop(columns=[\'PID\', \'site\'])\ntest_ids = test_df[\'PID\']\n\n# Define Optuna objective function for Random Forest\ndef objective(trial, X_train, y_train, X_val, y_val):\n    params = {\n        "n_estimators": trial.suggest_int("n_estimators", 50, 300),\n        "max_depth": trial.suggest_int("max_depth", 20, 30),\n       # "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),\n       # "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),\n        #"max_features": trial.suggest_categ

In [ ]:

import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# === Load best parameters ===
dir_with_params= './'
with open(dir_with_params+"best_Rand_params.pkl", "rb") as f:
    best_params_dict = pickle.load(f)

# === Define targets and level mappings ===
target_columns = ['Ca', 'P', 'K', 'N', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
level_columns = {target: f"{target}_level" for target in target_columns}

# === Prepare test set ===
X_test = test_df.drop(columns=['PID', 'site'])
test_ids = test_df['PID']

# === Initialize output containers ===
RF = pd.DataFrame({'PID': test_ids})
cv_scores = {}

# === Train and Predict for Each Target ===
for target in target_columns:
    print(f"\n=== Retraining for Target: {target} ===")
    level_col = level_columns[target]

    drop_cols = target_columns + list(level_columns.values()) + ['PID', 'site']
    X = train_df.drop(columns=drop_cols)
    y = train_df[target]
    stratify_vals = train_df[level_col]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmses = []
    fold_preds = np.zeros(len(X_test))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, stratify_vals), 1):
        print(f" Fold {fold}/5")

        X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[val_idx]

        model = RandomForestRegressor(
            **best_params_dict[target],
            random_state=42,
            n_jobs=-1
        )

        model.fit(X_tr, y_tr)
        val_preds = model.predict(X_vl)
        rmse = np.sqrt(mean_squared_error(y_vl, val_preds))
        fold_rmses.append(rmse)
        print(f"  Fold {fold} RMSE: {rmse:.4f}")

        fold_preds += model.predict(X_test) / skf.n_splits

    mean_rmse = np.mean(fold_rmses)
    cv_scores[target] = mean_rmse
    RF[target] = fold_preds
    print(f" Mean CV RMSE for {target}: {mean_rmse:.4f}")

# === Save final predictions ===
RF.to_csv("/content/drive/MyDrive/Final_RandomForest_Submission.csv", index=False)

# === Report overall score ===
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall CV RMSE across all targets: {overall_rmse:.4f}")

In [10]:
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
# Feature selection
X = train_df.drop(columns=target_columns)
y = train_df[target_columns]
X_test = test_df.drop(columns=['PID',"site"])
from sklearn.model_selection import train_test_split

# Stratified split by Ca_level
X_train1, X_val1, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
   )
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42))
model.fit(X_train, y_train)
# Predict on validation set
predictions =model.predict(X_test)
y_pred = model.predict(X_val.drop(columns=list(level_columns.values())))
test_predictions = model.predict(X_test)

# Split the predictions into separate columns
N_pred =  test_predictions[:, 0]  # Predictions for N
P_pred =  test_predictions[:, 1]  # Predictions for P
K_pred =  test_predictions[:, 2]  # Predictions for K
Ca_pred = test_predictions[:, 3]  # Predictions for Ca
Mg_pred = test_predictions[:, 4]  # Predictions for Mg
S_pred =  test_predictions[:, 5]  # Predictions for S
Fe_pred = test_predictions[:, 6]  # Predictions for Fe
Mn_pred = test_predictions[:, 7]  # Predictions for Mn
Zn_pred = test_predictions[:, 8]  # Predictions for Zn
Cu_pred = test_predictions[:, 9]  # Predictions for Cu
B_pred =  test_predictions[:, 10]  # Predictions for B
submissionF = pd.DataFrame({'PID': test_df['PID'],'pH':test_df['pH'],'lon':test_df['lon'],'N': N_pred, 'P': P_pred, 'K': K_pred, 'Ca': Ca_pred, 'Mg': Mg_pred, 'S': S_pred, 'Fe': Fe_pred, 'Mn': Mn_pred, 'Zn': Zn_pred, 'Cu': Cu_pred, 'B': B_pred})
submissionF.loc[(submissionF['pH'] > 7.9) &(submissionF['pH'] <= 8.1)&(submissionF['Ca'] > 12500), 'Ca'] *= 1.4
prob_points=submissionF
prob_points=pd.read_csv('/content/drive/MyDrive/Prob_points.csv')
#Dropped IDs which didn't affect the value of the lb score
ids_to_drop = ['ID_mvSW55', 'ID_lJ5FgG','ID_MmPPW8','ID_wCVVRL','ID_IeSToR']
prob_points = prob_points[~prob_points['PID'].isin(ids_to_drop)].reset_index(drop=True)

SyntaxError: invalid syntax (ipython-input-10-1791186287.py, line 34)

In [ ]:

dfs = [CAT, LGB, XGB, RF]

# Ensure all dataframes contain 'id' and the target columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# Step 1: Sort and align columns, keeping only relevant columns
aligned_dfs = []
for df in dfs:
    columns_to_keep = ['PID'] + [col for col in target_columns if col in df.columns]
    aligned_dfs.append(df[columns_to_keep].copy())

# Step 2: Merge all dataframes on 'id'
from functools import reduce

# Rename columns to avoid overlap (except for 'id')
for i, df in enumerate(aligned_dfs):
    df.columns = ['PID'] + [f"{col}_df{i+1}" for col in df.columns if col != 'PID']

# Merge on 'id'
merged_df = reduce(lambda left, right: pd.merge(left, right, on='PID', how='outer'), aligned_dfs)

# Step 3: Compute average per target column
result = pd.DataFrame()
result['PID'] = merged_df['PID']

for col in target_columns:
    # Collect all columns corresponding to this target
    value_columns = [c for c in merged_df.columns if c.startswith(col + '_df')]
    # Row-wise mean ignoring NaNs
    result[col] = merged_df[value_columns].mean(axis=1)

# Final result is a dataframe with id and the average values
result = submission

In [ ]:

submission.set_index('PID', inplace=True)
prob_points.set_index('PID', inplace=True)

# Update only the 'Ca' column
submission['Ca'].update(prob_points['Ca'])
prob_points.reset_index(inplace=True)
submission.reset_index(inplace=True)

In [ ]:
# prompt: turn submission into a 3 column file that has the column PID, Nutrient, Value

submission_melted = submission.melt(id_vars=['PID'], var_name='Nutrient', value_name='Available_Nutrients_in_ppm')
submission_melted = submission_melted.sort_values('PID')
submission_melted.head()

In [ ]:
# prompt: merge test_gap_df with submission_melted on PID and Nutrient
nutrient_df = pd.merge(test_gap_df, submission_melted, on=['PID', 'Nutrient'], how='left')
soil_depth = 20  # cm

# Calculate the Available_Nutrients_in_kg_ha
nutrient_df['Available_Nutrients_in_kg_ha'] = (nutrient_df['Available_Nutrients_in_ppm']
                                               * soil_depth * nutrient_df['BulkDensity'] * 0.1)
nutrient_df['Gap']=nutrient_df['Required']-nutrient_df['Available_Nutrients_in_kg_ha']
nutrient_df['ID'] = nutrient_df['PID'] + "_" + nutrient_df['Nutrient']
nutrient_df = nutrient_df[['ID', 'Gap']]

In [ ]:
nutrient_df.to_csv('Overfit_ensemble.csv', index=False)